In [32]:
from azure.storage.blob import BlobServiceClient
import pyarrow.parquet as pq

import os
from ultralytics import YOLO
from predictFn import PredictLicensePlate
import re
import cv2
from io import BytesIO
import base64
import numpy as np
import concurrent.futures
import pandas as pd

dirnameYolo=os.path.abspath("yoloBest.pt")
model = YOLO(os.path.join(dirnameYolo))
instance = PredictLicensePlate(model)

connectionString = "DefaultEndpointsProtocol=https;AccountName=iessimarroes01;AccountKey=E5B9vCkp4No1NdRh7xo/iBL5xEWQnEWXqAAzlkY8ZNPrVQGbcvcjH34Vs+OSdN9d1+/USTeZgTjQ+AStdd9j3w==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connectionString)

container_name = "cars"
container_client = blob_service_client.get_container_client(container_name)

blobs = container_client.list_blobs()


In [42]:
num_hilos = os.cpu_count()
print(num_hilos)

def procesar_linea(row):
    index,data = row
    image_b64 = data['photo']
    filestr = image_b64.split(',')[1]
    img_bytes = base64.b64decode(filestr)
    img_stream = BytesIO(img_bytes)
    imgArray = cv2.imdecode(np.frombuffer(img_stream.read(), np.uint8), cv2.IMREAD_COLOR)
    res = instance.doPredict(imgArray)
    print("Res "+str(res))
    return res

def procesar_dataframe(df):
    num_hilos = os.cpu_count()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_hilos) as executor:
        resultados = list(executor.map(procesar_linea, df.iterrows()))

    df['matricula'] = resultados
    return df

8


In [39]:
for blob in blobs:
    if not blob.name.endswith('.parquet'):
        continue

    blob_client = container_client.get_blob_client(blob.name)
    stream = BytesIO()
    blob_client.download_blob().download_to_stream(stream)
    stream.seek(0)

    parquet_file = pq.ParquetFile(stream)
    df = parquet_file.read().to_pandas()
    # list_license_plates = []
    #tmp
    df = df.head(100)
    df = procesar_dataframe(df)
    break
    # for index, row in df.iterrows():
    #     image_b64 = row['photo']
    #     filestr = image_b64.split(',')[1]
    #     img_bytes = base64.b64decode(filestr)
    #     img_stream = BytesIO(img_bytes)
    #     imgArray = cv2.imdecode(np.frombuffer(img_stream.read(), np.uint8), cv2.IMREAD_COLOR)
        
    #     res = instance.doPredict(imgArray)
    #     list_license_plates.append(res)
    #     # Opcionalmente, guardar la imagen en otro formato utilizando PIL
    #     #image_pil = Image.fromarray(imgArray)
    #     #image_pil.save(os.path.join('./', f"image_{index}.png"))
    #     print("Res "+str(res))
    
    # df['matricula'] = list_license_plates
    # df = df.drop('photo', axis=1)
    
    # print(df[df['matricula'] == False].shape[0])
    # print(df[df['matricula'] != False].shape[0])
    # break









Traceback (most recent call last):
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 166, in doPredict
    TabImgSelect =self.DetectLicenseWithYolov8(image)
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 42, in DetectLicenseWithYolov8
    results = self.model.predict(img)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\model.py", line 238, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\predictor.py", line 127, in __call__
    return list(self.stream_inference(source, model))  # merge list of Result into one
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 35, i

Excepcion
Res False


Traceback (most recent call last):
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 166, in doPredict
    TabImgSelect =self.DetectLicenseWithYolov8(image)
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 42, in DetectLicenseWithYolov8
    results = self.model.predict(img)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\model.py", line 238, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\predictor.py", line 127, in __call__
    return list(self.stream_inference(source, model))  # merge list of Result into one
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 35, in genera

Excepcion
Res False
Excepcion
Res False
Excepcion
Res False
Excepcion
Res False
Excepcion


Traceback (most recent call last):
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 166, in doPredict
    TabImgSelect =self.DetectLicenseWithYolov8(image)
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 42, in DetectLicenseWithYolov8
    results = self.model.predict(img)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\model.py", line 238, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\predictor.py", line 127, in __call__
    return list(self.stream_inference(source, model))  # merge list of Result into one
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 35, in genera

...

Res False
Excepcion
Res False
Excepcion
Res False
Excepcion
Res False
Excepcion
Res False


Traceback (most recent call last):
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 166, in doPredict
    TabImgSelect =self.DetectLicenseWithYolov8(image)
  File "c:\Users\AITOR\Desktop\PROYECTOS\cleanParquetNotebook\predictFn.py", line 42, in DetectLicenseWithYolov8
    results = self.model.predict(img)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\model.py", line 238, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "c:\Users\AITOR\anaconda3\lib\site-packages\ultralytics\yolo\engine\predictor.py", line 127, in __call__
    return list(self.stream_inference(source, model))  # merge list of Result into one
  File "c:\Users\AITOR\anaconda3\lib\site-packages\torch\utils\_contextlib.py", line 35, in genera

Excepcion
Res False
Excepcion
Excepcion
Res False
Res False


c:\Users\AITOR\anaconda3\lib\site-packages\skimage\transform\radon_transform.py:75: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


Matricula A 476
Matricula A 476
Matricula A 6RX
Matricula A 478
Matricula A 476
  Matricula detectada  confianza
0                 476          2
1                 476          1
3                 478          1
4                 476          1
First detection 476
Final detection 476
Res 476


In [40]:
print(df)

                                                photo                 enter  \
0   data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...            Albereda 2   
1   data:image/gif;base64,/9j/4AAQSkZJRgABAQAAAQAB...               Carme 1   
2   data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...     Academic Maravall   
3   data:image/gif;base64,/9j/4AAQSkZJRgABAQAAAQAB...               Maulets   
4   data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...  Corts Valencianes 1A   
..                                                ...                   ...   
95  data:image/gif;base64,/9j/4AAQSkZJRgABAQAAAQAB...               Maulets   
96  data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...                N-340A   
97  data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...                N-340B   
98  data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...               CV-645A   
99  data:image/gif;base64,/9j/4AAQSkZJRgABAQEBLAEs...  Corts Valencianes 2B   

                    exit                      date 